## R2 To R1 Point Attractor, S1 Damping, R1 Sphere Avoidance

In [1]:
using PBDS, StaticArrays, LinearAlgebra, BenchmarkTools

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-12983492175206942648\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-17863049939896729247\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-12983492175206942648\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

## Setup

### Point Distance Attractor

In [2]:
PBDS.metric_chart(xn, task::Attractor{<:DistanceFromPoint{R2,R1,S}}, CN::Chart{1,R1}) where S = 
    default_metric(xn, task, CN)
PBDS.potential_chart(xn, task::Attractor{<:DistanceFromPoint{R2,R1,S}}, CN::Chart{1,R1}) where S = xn[1]^2
PBDS.dissipative_forces_chart(xn, vn, task::Attractor{<:DistanceFromPoint{R2,R1,S}}, CN::Chart{1,R1}) where S =  -3*vn
PBDS.weight_metric_chart(xn, vn, task::Attractor{<:DistanceFromPoint{R2,R1,S}}, CN::Chart{1,R1}) where S = 
    default_weight_metric(xn, vn, task, CN)

### Angular Damping Around Goal

In [14]:
PBDS.default_coord_rep(::Damping{<:AngularPositionAroundPoint{R2,S1,S}}) where S = EmbRep()
PBDS.metric_emb(xne, task::Damping{<:AngularPositionAroundPoint{R2,S1,S}}) where S =
    default_metric(xne, task)
PBDS.potential_emb(xne, task::Damping{<:AngularPositionAroundPoint{R2,S1,S}}) where S = 0.
PBDS.dissipative_forces_emb(xne, vne, task::Damping{<:AngularPositionAroundPoint{R2,S1,S}}) where S = -2*vne
PBDS.weight_metric_emb(xne, vne, task::Damping{<:AngularPositionAroundPoint{R2,S1,S}}) where S =
    default_weight_metric(xne, vne, task)

### Circle Avoidance

In [4]:
function PBDS.metric_chart(xn, task::CollisionAvoidance{<:DistanceFromSphereSurface{R2,R1,S}}, CN::Chart{1,R1}) where S
    ψx = exp(1.e1 / xn[1]^2)
    G = SMatrix{1,1,eltype(xn)}([ψx])
end
PBDS.potential_chart(xn, task::CollisionAvoidance{<:DistanceFromSphereSurface{R2,R1,S}}, CN::Chart{1,R1}) where S = 0.
PBDS.dissipative_forces_chart(xn, vn, task::CollisionAvoidance{<:DistanceFromSphereSurface{R2,R1,S}}, CN::Chart{1,R1}) where S = 0*vn
function PBDS.weight_metric_chart(xn, vn, task::CollisionAvoidance{<:DistanceFromSphereSurface{R2,R1,S}}, CN::Chart{1,R1}) where S
    offset_distance = 5.
    λ = (xn[1] < offset_distance && vn[1] < 0.) ? 1. : 0.
    W = SMatrix{1,1,eltype(xn)}(I)*λ
end

In [5]:
xm_goal = SA[-8., 3.]
M = R2
CM = Chart{1,M}()
tasks, CNs = TaskList(), ChartList()

N = R1
push!(tasks, Attractor(DistanceFromPoint{M,N}(xm_goal)))
push!(CNs, Chart{1,N}())

N = S1
push!(tasks, Damping(AngularPositionAroundPoint{M,N}(xm_goal)))
push!(CNs, Chart{Angleπ,N}())

N = R1
CN = Chart{1,N}()
centers, radii = [], []
push!(centers, SA[-4., 0.])
push!(radii, 1.)
push!(centers, SA[-3., 3.])
push!(radii, 1.)
push!(centers, SA[-6., 2.])
push!(radii, 1.)

for i in 1:length(centers)
    push!(tasks, CollisionAvoidance(DistanceFromSphereSurface{M,N}(centers[i], radii[i])))
    push!(CNs, CN)
end

## Point Acceleration

In [6]:
# Initial state
xm = SA[3., 4.]
vm = SA[1., 1.]
σxddot, = multiple_task_acceleration(xm, vm, tasks, CM, CNs)

([-25.090029561945727, -4.829346949744661], Chart[])

## Single Trajectory

In [9]:
Time = 8
dt = 0.05

traj = propagate_tasks(xm, vm, tasks, CM, CNs, Time, dt)
traj.xm[end]

2-element SArray{Tuple{2},Float64,1,2} with indices SOneTo(2):
 -7.986251764430824
  3.007570407257694

## Multiple Trajectories

In [10]:
using Makie, Observables, ProgressMeter, VectorizedRoutines

In [15]:
Time = 8
dt = 0.01
xlim, Δx = 1., 1.
vlim, Δv = 2., 2.

xm0 = Array{Any}(undef, 2)
vm0 = Array{Any}(undef, 2)

xm0[1], xm0[2] = Matlab.meshgrid(-xlim:Δx:xlim, -xlim:Δx:xlim)
vm0[1], vm0[2] = Matlab.meshgrid(-vlim:Δv:vlim, -vlim:Δv:vlim)

for a in (xm0, vm0), i in 1:2
    a[i] = reshape(a[i], length(a[i]))
end

nx = length(xm0[1])
nv = length(vm0[1])
ix, iv = Matlab.meshgrid(1:nx, 1:nv)
ix = reshape(ix, length(ix))
iv = reshape(iv, length(iv))

ntraj = nx*nv
trajs = Array{Any}(undef, ntraj)
xm0 = [xm0[1]'; xm0[2]']
vm0 = [vm0[1]'; vm0[2]']
p = Progress(ntraj)

Threads.@threads for i in 1:ntraj
    trajs[i] = propagate_tasks(SA[xm0[:,ix[i]]...], SA[vm0[:,iv[i]]...], tasks, CM, CNs, Time, dt)
    next!(p)
end

Progress: 100%|█████████████████████████████████████████| Time: 0:00:06


In [17]:
fxx(i) = [trajs[j].xm[i][1] for j = 1:ntraj]
fxy(i) = [trajs[j].xm[i][2] for j = 1:ntraj]
iobs = Observable(1)

ax_size, plot_size = 6, 500
scene = Scene(resolution = (plot_size, plot_size))
Makie.scatter!(scene, lift(i -> fxx(i), iobs), lift(i -> fxy(i), iobs), markersize = ax_size/20., color = :blue)

for i in 1:length(centers)
    Makie.scatter!(scene, centers[i]', markersize = 2*radii[i], strokecolor = :black, strokewidth=1.5, color = :orange)
end
Makie.scatter!(scene, xm_goal', markersize = ax_size/20, color = :darkgreen)

for i = 1:ntraj
    Makie.lines!(scene, getindex.(trajs[i].xm,1), getindex.(trajs[i].xm,2), color = :purple)
end

Makie.xlims!(scene, (-ax_size-4, ax_size-4))
Makie.ylims!(scene, (-ax_size+2, ax_size+2))
Makie.xlabel!(scene, "x")
Makie.ylabel!(scene, "y")
axis = scene[Axis]
display(scene)

GLMakie.Screen(...)

In [18]:
function record_scene(scene, filename, iobs, N)
    p = Progress(N)
    record(scene, filename, 1:N, framerate = 60) do i
        iobs[] = i
        next!(p)
    end
    display("text/html", html_video(filename))
end

filename = "R2_To_R1PointDistanceAttractor_S1Damping_R1SphereAvoidance.mp4"
record_scene(scene, filename, iobs, length(trajs[1].xm))

Progress: 100%|█████████████████████████████████████████| Time: 0:00:17
